In [ ]:
!pip install praw
!pip install psaw
!pip install pmaw

     |████████████████████████████████| 176 kB 5.2 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import praw
from praw.models import MoreComments
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk import FreqDist
#import emoji
import re
import seaborn as sns
import pmaw
from pmaw import PushshiftAPI
import timeit
import requests

print('Libraries Imported')

Libraries Imported


In [ ]:
reddit = praw.Reddit(client_id = 'bNcar4az2yBMIJ8yy6QiGQ', client_secret = 'PJlkh_souAGbKcq1ombUtToVod1Npg', user_agent = 'ew_artistfreq')
print('API Authenticated')

API Authenticated


In [ ]:
api = PushshiftAPI()

https://api.pushshift.io/reddit/search/submission/?q=Drake&after=2010-01-01&before=2020-12-31&subreddit=Music

https://api.pushshift.io/reddit/search/submission/?q=Taylor%20Swift&after=2010-01-01&before=2020-12-31&subreddit=Music

https://api.pushshift.io/reddit/search/submission/?q=The%20Weeknd&after=2010-01-01&before=2020-12-31&subreddit=Music

https://api.pushshift.io/reddit/search/submission/?q=Ariana%20Grande&after=2010-01-01&before=2020-12-31&subreddit=Music

https://api.pushshift.io/reddit/search/submission/?q=J.%20Cole&after=2010-01-01&before=2020-12-31&subreddit=Music

https://api.pushshift.io/reddit/search/submission/?q=Glee%20Cast&after=2010-01-01&before=2020-12-31&subreddit=Music

In [ ]:
import json
import csv
import datetime


In [ ]:
#Adapted from this https://gist.github.com/dylankilkenny/3dbf6123527260165f8c5c3bc3ee331b
#This function builds an Pushshift URL, accesses the webpage and stores JSON data in a nested list
def getPushshiftData(query, after, before, sub):
    #Build URL
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=10000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    #Print URL to show user
    print(url)
    #Request URL
    r = requests.get(url)
    #Load JSON data from webpage into data variable
    data = json.loads(r.text)
    #return the data element which contains all the submissions data
    return data['data']

In [ ]:
#This function will be used to extract the key data points from each JSON result
def collectSubData(subm):
    #subData was created at the start to hold all the data which is then added to our global subStats dictionary.
    subData = list() #list to store data points
    title = subm['title']
    url = subm['url']
    #flairs are not always present so we wrap in try/except
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']

    #Put all data points into a tuple and append to subData
    subData.append((sub_id,title,url,author,score,created,numComms,permalink,flair))
    #Create a dictionary entry of current submission data and store all data related to it
    subStats[sub_id] = subData

In [ ]:
#Create your timestamps and queries for your search URL
after = "2010-01-01" 
before = "2020-12-31" 
query = "Drake" 
sub = "Music"

#subCount tracks the no. of total submissions we collect
subCount = 0
#subStats is the dictionary where we will store our data.
subStats = {}

In [ ]:
data = getPushshiftData(query, after, before, sub)
# Will run until all posts have been gathered i.e. When the length of data variable = 0
# from the 'after' date up until before date
while len(data) > 0: #The length of data is the number submissions (data[0], data[1] etc), once it hits zero (after and before vars are the same) end
    try:
      for submission in data:
          collectSubData(submission)
          subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
      print(len(data))
      print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    #update after variable to last created date of submission
      after = data[-1]['created_utc']
    #data has changed due to the new after variable provided by above code
      data = getPushshiftData(query, after, before, sub)
    except:
      pass
print(len(data))

https://api.pushshift.io/reddit/search/submission/?title=Drake&size=10000&after=2010-01-01&before=2020-12-31&subreddit=Music
100
2010-07-16 11:24:42
https://api.pushshift.io/reddit/search/submission/?title=Drake&size=10000&after=1279279482&before=2020-12-31&subreddit=Music
100
2010-11-17 14:15:33
https://api.pushshift.io/reddit/search/submission/?title=Drake&size=10000&after=1290003333&before=2020-12-31&subreddit=Music
100
2011-06-28 14:27:38
https://api.pushshift.io/reddit/search/submission/?title=Drake&size=10000&after=1309271258&before=2020-12-31&subreddit=Music
100
2011-11-07 15:08:20
https://api.pushshift.io/reddit/search/submission/?title=Drake&size=10000&after=1320678500&before=2020-12-31&subreddit=Music
100
2011-12-18 12:20:55
https://api.pushshift.io/reddit/search/submission/?title=Drake&size=10000&after=1324210855&before=2020-12-31&subreddit=Music
100
2012-02-05 03:47:37
https://api.pushshift.io/reddit/search/submission/?title=Drake&size=10000&after=1328413657&before=2020-12-

In [ ]:
def updateSubs_file():
    upload_count = 0
    print("input filename of submission file, please add .csv")
    filename = input() #This asks the user what to name the file
    file = filename
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["Post ID","Title","Url","Author","Score","Publish Date","Total No. of Comments","Permalink","Flair"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded")
updateSubs_file()

input filename of submission file, please add .csv
glee.csv
309 submissions have been uploaded


In [ ]:
#Create your timestamps and queries for your search URL
after = "2010-01-01" 
before = "2020-12-31" 
query = "Taylor Swift" #Keyword(s) to look for in submissions
sub = "Music" #Which Subreddit to search in

#subCount tracks the no. of total submissions we collect
subCount = 0
#subStats is the dictionary where we will store our data.
subStats = {}
data = getPushshiftData(query, after, before, sub)
# Will run until all posts have been gathered i.e. When the length of data variable = 0
# from the 'after' date up until before date
while len(data) > 0: #The length of data is the number submissions (data[0], data[1] etc), once it hits zero (after and before vars are the same) end
    try:
      for submission in data:
          collectSubData(submission)
          subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
      print(len(data))
      print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    #update after variable to last created date of submission
      after = data[-1]['created_utc']
    #data has changed due to the new after variable provided by above code
      data = getPushshiftData(query, after, before, sub)
    except:
      pass
print(len(data))

In [ ]:
updateSubs_file()

input filename of submission file, please add .csv
taylor_reddit.csv
3512 submissions have been uploaded


In [ ]:
#Create your timestamps and queries for your search URL
after = "2010-01-01"
before = "2020-12-31" 
query = "J. Cole" #Keyword(s) to look for in submissions
sub = "Music" #Which Subreddit to search in

#subCount tracks the no. of total submissions we collect
subCount = 0
#subStats is the dictionary where we will store our data.
subStats = {}
data = getPushshiftData(query, after, before, sub)
# Will run until all posts have been gathered i.e. When the length of data variable = 0
# from the 'after' date up until before date
while len(data) > 0: #The length of data is the number submissions (data[0], data[1] etc), once it hits zero (after and before vars are the same) end
    try:
      for submission in data:
          collectSubData(submission)
          subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
      print(len(data))
      print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    #update after variable to last created date of submission
      after = data[-1]['created_utc']
    #data has changed due to the new after variable provided by above code
      data = getPushshiftData(query, after, before, sub)
    except:
      pass
print(len(data))

In [ ]:
updateSubs_file()

input filename of submission file, please add .csv
jcole_reddit.csv
872 submissions have been uploaded


In [ ]:
#Create your timestamps and queries for your search URL
after = "2010-01-01" 
before = "2020-12-31" 
query = "Glee" #Keyword(s) to look for in submissions
sub = "Music" #Which Subreddit to search in

#subCount tracks the no. of total submissions we collect
subCount = 0
#subStats is the dictionary where we will store our data.
subStats = {}
data = getPushshiftData(query, after, before, sub)
# Will run until all posts have been gathered i.e. When the length of data variable = 0
# from the 'after' date up until before date
while len(data) > 0: #The length of data is the number submissions (data[0], data[1] etc), once it hits zero (after and before vars are the same) end
    try:
      for submission in data:
          collectSubData(submission)
          subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
      print(len(data))
      print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    #update after variable to last created date of submission
      after = data[-1]['created_utc']
    #data has changed due to the new after variable provided by above code
      data = getPushshiftData(query, after, before, sub)
    except:
      pass
print(len(data))

https://api.pushshift.io/reddit/search/submission/?title=Glee&size=10000&after=2010-01-01&before=2020-12-31&subreddit=Music
100
2011-03-21 02:53:54
https://api.pushshift.io/reddit/search/submission/?title=Glee&size=10000&after=1300676034&before=2020-12-31&subreddit=Music
100
2012-04-21 23:13:12
https://api.pushshift.io/reddit/search/submission/?title=Glee&size=10000&after=1335049992&before=2020-12-31&subreddit=Music
100
2020-08-08 22:19:27
https://api.pushshift.io/reddit/search/submission/?title=Glee&size=10000&after=1596925167&before=2020-12-31&subreddit=Music
9
2020-12-24 20:34:02
https://api.pushshift.io/reddit/search/submission/?title=Glee&size=10000&after=1608842042&before=2020-12-31&subreddit=Music
0


In [ ]:
updateSubs_file()

NameError: ignored

In [ ]:
#Create your timestamps and queries for your search URL
after = "2010-01-01" 
before = "2020-12-31" 
query = "The Weeknd" #Keyword(s) to look for in submissions
sub = "Music" #Which Subreddit to search in

#subCount tracks the no. of total submissions we collect
subCount = 0
#subStats is the dictionary where we will store our data.
subStats = {}
data = getPushshiftData(query, after, before, sub)
# Will run until all posts have been gathered i.e. When the length of data variable = 0
# from the 'after' date up until before date
while len(data) > 0: #The length of data is the number submissions (data[0], data[1] etc), once it hits zero (after and before vars are the same) end
    try:
      for submission in data:
          collectSubData(submission)
          subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
      print(len(data))
      print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    #update after variable to last created date of submission
      after = data[-1]['created_utc']
    #data has changed due to the new after variable provided by above code
      data = getPushshiftData(query, after, before, sub)
    except:
      pass
print(len(data))

https://api.pushshift.io/reddit/search/submission/?title=The Weeknd&size=1000&after=2010-01-01&before=2020-12-31&subreddit=Music
100
2011-12-21 05:33:09
https://api.pushshift.io/reddit/search/submission/?title=The Weeknd&size=1000&after=1324445589&before=2020-12-31&subreddit=Music
100
2012-04-10 02:29:10
https://api.pushshift.io/reddit/search/submission/?title=The Weeknd&size=1000&after=1334024950&before=2020-12-31&subreddit=Music
100
2012-11-05 21:31:05
https://api.pushshift.io/reddit/search/submission/?title=The Weeknd&size=1000&after=1352151065&before=2020-12-31&subreddit=Music
100
2014-08-11 16:59:53
https://api.pushshift.io/reddit/search/submission/?title=The Weeknd&size=1000&after=1407776393&before=2020-12-31&subreddit=Music
100
2015-05-26 05:21:35
https://api.pushshift.io/reddit/search/submission/?title=The Weeknd&size=1000&after=1432617695&before=2020-12-31&subreddit=Music
100
2015-08-03 17:55:31
https://api.pushshift.io/reddit/search/submission/?title=The Weeknd&size=1000&afte

In [ ]:
updateSubs_file()

input filename of submission file, please add .csv
weeknd_reddit.csv
2245 submissions have been uploaded


In [ ]:
#Create your timestamps and queries for your search URL
after = "2010-01-01" 
before = "2020-12-31" 
query = "Ariana Grande" #Keyword(s) to look for in submissions
sub = "Music" #Which Subreddit to search in

#subCount tracks the no. of total submissions we collect
subCount = 0
#subStats is the dictionary where we will store our data.
subStats = {}
data = getPushshiftData(query, after, before, sub)
# Will run until all posts have been gathered i.e. When the length of data variable = 0
# from the 'after' date up until before date
while len(data) > 0: #The length of data is the number submissions (data[0], data[1] etc), once it hits zero (after and before vars are the same) end
    try:
      for submission in data:
          collectSubData(submission)
          subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
      print(len(data))
      print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    #update after variable to last created date of submission
      after = data[-1]['created_utc']
    #data has changed due to the new after variable provided by above code
      data = getPushshiftData(query, after, before, sub)
    except:
      pass
print(len(data))

https://api.pushshift.io/reddit/search/submission/?title=Ariana Grande&size=1000&after=2010-01-01&before=2020-12-31&subreddit=Music
100
2014-10-21 13:58:52
https://api.pushshift.io/reddit/search/submission/?title=Ariana Grande&size=1000&after=1413899932&before=2020-12-31&subreddit=Music
100
2015-06-13 08:11:53
https://api.pushshift.io/reddit/search/submission/?title=Ariana Grande&size=1000&after=1434183113&before=2020-12-31&subreddit=Music
100
2016-02-09 10:43:58
https://api.pushshift.io/reddit/search/submission/?title=Ariana Grande&size=1000&after=1455014638&before=2020-12-31&subreddit=Music
100
2016-05-24 23:47:40
https://api.pushshift.io/reddit/search/submission/?title=Ariana Grande&size=1000&after=1464133660&before=2020-12-31&subreddit=Music
100
2016-10-10 18:17:15
https://api.pushshift.io/reddit/search/submission/?title=Ariana Grande&size=1000&after=1476123435&before=2020-12-31&subreddit=Music
100
2017-05-02 22:07:12
https://api.pushshift.io/reddit/search/submission/?title=Ariana 

In [ ]:
updateSubs_file()

input filename of submission file, please add .csv
ariana_reddit.csv
1685 submissions have been uploaded
